In [2]:
# UTA-PL1 with OR-Tools, only linear utilities and no interpolation

from ortools.linear_solver import pywraplp
import numpy as np

alts = ["A","B","C","D","E"]
# three criteria (e.g., perf, reliability, battery)
G = np.array([
    [60, 50, 100],  # A
    [40, 100, 40],  # B
    [100, 20, 80],  # C
    [20, 80, 30],   # D
    [0,  0,  0],    # E
], dtype=float)


# Normalize each criterion to [0,1] (monotone increasing)
g_min = G.min(axis=0)
g_max = G.max(axis=0)

U_normalized = (G - g_min) / (g_max - g_min + 1e-12)  # shape (5 alts, 3 crit)



# ----- LP: find weights p >=0, sum p = 1, s.t. A>B>C>D>E -----
solver = pywraplp.Solver.CreateSolver('GLOP')
p1 = solver.NumVar(0.0, 1.0, "p1")
p2 = solver.NumVar(0.0, 1.0, "p2")
p3 = solver.NumVar(0.0, 1.0, "p3")

p = [p1, p2, p3]

solver.Add(sum(p) == 1.0)

# Utilities U(a) = sum_i p_i * u_i(g_i(a))
U = [sum(p[i] * U_normalized[alt, i] for i in range(3)) for alt in range(5)]

eps = 1e-3
# Erreurs par alternative
sigma = [solver.NumVar(0.0, solver.infinity(), f"sigma_{a}") for a in range(5)]

# A > B > C > D > E
solver.Add(U[0] >= U[1] + eps - (sigma[0] + sigma[1]))  # A > B
solver.Add(U[1] >= U[2] + eps - (sigma[1] + sigma[2]))  # B > C
solver.Add(U[2] >= U[3] + eps - (sigma[2] + sigma[3]))  # C > D
solver.Add(U[3] >= U[4] + eps - (sigma[3] + sigma[4]))  # D > E

# Objectif : minimiser la somme des écarts
solver.Minimize(sum(sigma))

status = solver.Solve()
assert status == pywraplp.Solver.OPTIMAL, "Ranking infeasible with given data."

print("Weights:", [pi.solution_value() for pi in p])

for a, name in enumerate(alts):
    print(f"U({name}) = {U[a].solution_value():.3f}")
print("Sigmas:", [s.solution_value() for s in sigma])

print("Total F* =", sum(s.solution_value() for s in sigma))

Weights: [0.14285714285714302, 0.4925974025974026, 0.3645454545454544]
U(A) = 0.697
U(B) = 0.696
U(C) = 0.533
U(D) = 0.532
U(E) = 0.000
Sigmas: [0.0, 0.0, 0.0, 0.0, 0.0]
Total F* = 0.0
